<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-04-30 14:41:28
-------------------
qualified stocks: 93
with latest results: 87
still star stocks: 46
-------------------
Initial Investment:  1.10 C
CY Investment:  1.37 C
Reserve:  2.00 K
Current:  1.17 C
-------------------
Today PnL: -95.78 K (-0.81%)
Current PnL: -20.41 L (-14.9%)
CY Booked + Current PnL: -18.88 L (-13.78%)
-------------------
Total profit:  4.18 L
Total loss:  -24.60 L
-------------------
Total Booked + Current PnL: 8.10 L (7.36%)
Total Booked PnL: 28.52 L (25.91%)
Curr Year Booked PnL: 1.54 L (1.32%)
Prev Year Booked PnL: 26.98 L (24.52%)
Est FTT:  1.96 C
Est FTT PnL: 79.15 L (67.73%)
-------------------
Est LTT:  2.57 C
Est LTT PnL: 1.40 C (119.51%)
Deployed:  1.10 C
Current:  1.17 C
CAGR/XIRR %: 4.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HDFCBANK,1561.04,2170.00,4.66,X-LC,78.49,101972.0,19237.0,13042.0,0.80,23.25,12.79,39.01,4.0,1.48,0.87,34.73,X40,BTT,BANKS
71,SBIN,760.30,863.00,-17.80,M-LC,49.46,215326.0,13846.0,13372.0,-0.59,6.87,6.21,13.51,93.0,1.04,1.84,17.97,XY25,NTT,BANKS
70,SBILIFE,1496.49,1924.99,-0.20,H-LC,92.47,148100.0,22395.0,13596.0,1.91,17.82,9.18,28.63,NaN,1.65,1.27,31.94,AR,ATH,INSURANCE
63,RELIANCE,1288.53,1526.00,4.47,X-LC,88.17,222638.0,19050.0,18479.0,0.67,9.36,8.30,18.43,67.0,1.03,1.91,21.28,XY25,NTT,REFINERIES
24,DMART,3484.00,5112.00,-17.84,X-LC,64.52,101659.0,18043.0,21033.0,-1.72,21.58,20.69,46.73,33.0,0.86,0.87,24.44,AR,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ASIANTILES,75.41,137.00,282.13,M-SC,46.24,56341.0,-37469.0,114091.0,4.33,-39.94,202.50,81.67,-16.0,-0.33,0.48,9.11,XR,NTT,CERAMICS
33,HDFCLIFE,562.05,897.00,0.60,H-LC,68.82,148140.0,35730.0,31258.0,3.59,31.79,21.10,59.59,16.0,1.14,1.27,39.06,X40,BTT,INSURANCE
20,COFFEEDAY,59.14,80.00,-129.58,L-SC,93.55,61517.0,-52032.0,92085.0,1.97,-45.82,149.69,35.27,NaN,-0.57,0.53,50.56,XR,NTT,HOTELS
51,MEDANTA,1022.60,1486.00,4.56,X-MC,18.28,182506.0,27071.0,43363.0,1.94,17.42,23.76,45.32,8.0,0.62,1.56,24.18,XY24,NTT,HEALTHCARE
70,SBILIFE,1496.49,1924.99,-0.20,H-LC,92.47,148100.0,22395.0,13596.0,1.91,17.82,9.18,28.63,NaN,1.65,1.27,31.94,AR,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.0,-254.52,H-SC,79.57,83850.0,-49436.0,134026.0,-4.13,-37.09,159.84,63.47,-27.0,-0.37,0.72,40.55,XR,NTT,CHEMICALS
27,GICRE,335.06,583.0,2.32,H-MC,36.56,181770.0,35349.0,72999.0,-4.10,24.14,40.16,74.00,109.0,0.48,1.56,32.93,XY24,BTT,INSURANCE
77,STARHEALTH,551.56,761.0,-31.76,M-SC,86.02,139410.0,-60806.0,136831.0,-3.55,-30.37,98.15,37.97,-19.0,-0.44,1.19,11.94,XY24,NTT,INSURANCE
45,KANSAINER,299.63,340.0,-66.94,H-SC,77.42,231750.0,-37917.0,74253.0,-3.53,-14.06,32.04,13.47,-30.0,-0.51,1.98,16.00,XY24,NTT,PAINTS
81,TATAMOTORS,798.82,1065.0,-58.48,M-LC,11.83,131825.0,-31933.0,86504.0,-3.39,-19.50,65.62,33.32,-49.0,-0.37,1.13,10.92,XY24,NTT,AUTO


In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NHPC,82.08,115.00,217.46,M-MC,61.29,148641.0,7628.0,48933.0,-0.29,5.41,32.92,40.11,-54.0,0.16,1.27,21.52,XY24,NTT,POWER
37,ICICIPRULI,604.56,791.05,4.26,H-MC,72.04,101766.0,1409.0,29553.0,1.29,1.40,29.04,30.85,103.0,0.05,0.87,14.25,X40,ATH,INSURANCE
27,GICRE,335.06,583.00,2.32,H-MC,36.56,181770.0,35349.0,72999.0,-4.10,24.14,40.16,74.00,109.0,0.48,1.56,32.93,XY24,BTT,INSURANCE
18,CAMS,3643.00,5250.99,-1.55,H-SC,65.59,109505.0,7501.0,37527.0,-2.22,7.35,34.27,44.14,261.0,0.20,0.94,28.12,XR,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,-5.49,M-LC,52.69,166688.0,15482.0,50190.0,0.02,10.24,30.11,43.43,NaN,0.31,1.43,15.54,XY24,NTT,MISC


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-1.55,H-SC,65.59,109505.0,7501.0,37527.0,-2.22,7.35,34.27,44.14,261.0,0.20,0.94,28.12,XR,ATH,MISC
12,BANKINDIA,112.90,190.00,-26.97,M-MC,58.06,102555.0,2300.0,66168.0,-1.98,2.29,64.52,68.29,-12.0,0.03,0.88,27.83,XR,NTT,BANKS
91,VOLTAS,1272.00,1929.20,-20.53,H-MC,3.23,135289.0,-4631.0,76925.0,-2.74,-3.31,56.86,51.67,NaN,-0.06,1.16,1.32,AR,ATH,AC
1,AARTIIND,487.04,919.00,13.65,M-SC,74.19,70491.0,-10845.0,82982.0,-2.02,-13.33,117.72,88.69,-8.0,-0.13,0.60,18.87,XR,NTT,CHEMICALS
76,SONACOMS,578.05,1006.00,-35.78,M-MC,55.91,84149.0,-17010.0,91899.0,-2.24,-16.81,109.21,74.03,-8.0,-0.19,0.72,18.26,AR,BTT,AUTO


In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HDFCBANK,1561.04,2170.00,4.66,X-LC,78.49,101972.0,19237.0,13042.0,0.80,23.25,12.79,39.01,4.0,1.48,0.87,34.73,X40,BTT,BANKS
12,BANKINDIA,112.90,190.00,-26.97,M-MC,58.06,102555.0,2300.0,66168.0,-1.98,2.29,64.52,68.29,-12.0,0.03,0.88,27.83,XR,NTT,BANKS
86,UJJIVANSFB,52.77,60.00,-3.30,M-SC,98.92,115209.0,-27270.0,46786.0,-2.80,-19.14,40.61,13.70,-27.0,-0.58,0.99,35.98,X40N,NTT,BANKS
33,HDFCLIFE,562.05,897.00,0.60,H-LC,68.82,148140.0,35730.0,31258.0,3.59,31.79,21.10,59.59,16.0,1.14,1.27,39.06,X40,BTT,INSURANCE
70,SBILIFE,1496.49,1924.99,-0.20,H-LC,92.47,148100.0,22395.0,13596.0,1.91,17.82,9.18,28.63,NaN,1.65,1.27,31.94,AR,ATH,INSURANCE


In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1354.50,1800.0,-35.67,H-MC,100.00,47680.0,-29526.0,54918.0,-0.05,-38.24,115.18,32.89,-21.0,-0.54,0.41,31.31,XR,NTT,BANKS
86,UJJIVANSFB,52.77,60.0,-3.30,M-SC,98.92,115209.0,-27270.0,46786.0,-2.80,-19.14,40.61,13.70,-27.0,-0.58,0.99,35.98,X40N,NTT,BANKS
92,WHIRLPOOL,1112.81,2270.0,-35.54,M-SC,97.85,115137.0,14984.0,89162.0,-0.94,14.96,77.44,103.99,34.0,0.17,0.99,38.80,X40,NTT,DURABLES
78,SURYODAY,189.78,240.0,-37.74,H-SC,96.77,101120.0,-48806.0,88480.0,-1.93,-32.55,87.50,26.46,-41.0,-0.55,0.87,29.54,XR,NTT,BANKS
64,REPCOHOME,515.07,880.0,-12.33,H-SC,95.70,162464.0,-42534.0,187776.0,1.73,-20.75,115.58,70.85,-44.0,-0.23,1.39,31.15,XY24,NTT,FINANCE


In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.78
1,25,41.59
2,50,70.33


In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    37.97
X40     15.61
X40N    12.83
AR      12.37
XY25     9.69
XR       9.17
X200     1.18
SR       1.17
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    20.38
H-SC    19.43
M-SC    18.37
M-MC     7.58
H-MC     7.53
M-LC     6.81
H-LC     6.58
X-MC     6.55
X-SC     2.59
L-SC     2.37
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,12.26,-8.94,61.48
FMCG,10.22,-1.57,33.03
INSURANCE,8.56,4.02,34.67
IT,7.80,-33.00,99.28
FINANCE,7.61,-24.40,80.53
MISC,6.39,-38.13,88.11
PAINTS,5.91,-19.05,54.29
ELECTRICAL,4.27,-18.30,68.43
CHEMICALS,4.05,-14.18,89.43


In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3245397.0,29
X40N,1237146.0,16
XR,1165688.0,14
AR,1046841.0,12
X40,768619.0,13
SR,208117.0,2
XY25,163281.0,6
X200,79683.0,1


In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2240878.0,19
M-SC,2213893.0,22
X-LC,862303.0,14
M-MC,656952.0,7
H-MC,410041.0,7
X-SC,374554.0,4
L-SC,319975.0,4
M-LC,260917.0,5
X-MC,253570.0,5


In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1030808.0      8
H-SC       XY24       945739.0      6
           X40N       555784.0      6
M-MC       XY24       472831.0      4
M-SC       XR         451649.0      5
X-LC       X40        415394.0      6
H-SC       AR         392486.0      2
M-SC       X40N       386894.0      5
H-SC       XR         298872.0      4
M-LC       XY24       247545.0      4
H-MC       XY24       227542.0      3
X-LC       AR         187846.0      3
L-SC       XR         163364.0      2
M-SC       X40        162639.0      2
X-SC       XR         130717.0      1
           SR         122536.0      1
           X40N       121301.0      2
X-MC       X40N       117112.0      2
H-LC       AR         100784.0      2
X-LC       XY24        96712.0      1
M-SC       AR          96322.0      1
M-MC       AR          91899.0      1
L-SC       XY24        88452.0      1
M-SC       SR          85581.0      1
X-LC       XY25        82668.0      3
           X200        79683.0      1
H-MC       AR          76925.0      1
L-SC       AR          68159.0      1
M-MC       XR          66168.0      1
X-MC       X40         60675.0      1
H-LC       X40N        56055.0      1
H-MC       XR          54918.0      1
           X40         50656.0      2
H-SC       X40         47997.0      1
X-MC       XY24        43363.0      1
H-LC       XY25        41187.0      1
X-MC       AR          32420.0      1
H-LC       X40         31258.0      1
M-MC       XY25        26054.0      1
M-LC       XY25        13372.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
